## Import Libraries 

In [3]:
import sys
sys.path.append('/paths_generator.py')
import paths_generator as pg
import random as rand
import math as m
import numpy as np

## Simulated Annealing Function

In [1]:
def simulated_annealing (init_st, cost_func, max_it, max_T, cooling_rate):   #cooling_rate howa li ki khli T decreasa
  current_st = init_st
  best_st = init_st
  T = max_T

  for i in range(max_it):
    if T <= 0:   
      break

    new_st = current_st.get_neighbor()   # Get a random neighbor of the current state
    current_C = cost_func(current_st)
    new_C = cost_func(new_C)

    if new_C < current_C:
      current_st = new_st # Accept the new state as the current state
      if new_C < cost_func(best_st):
        best_state = new_st  # Update the best state if the new state has a minimal cost
    
    else: 
      proba = m.exp(- (current_C - new_C) / T)
      if rand.random() < proba:
        current_st = new_st
        T *= cooling_rate # Cool down the temperature for the next iteration

  return best_st

## Utils

Init data

In [ ]:
T_Ci, assets, slr, region, entries, rows, cols= pg.generator()    # region is a list?  # entries == Bad zones headers
zones_num = pg.region.shape[0]*pg.region.shape[1]

 Check false

In [1]:
def check_false(lst):

    for element in lst:
        
        if not element:
            return False
        
    return True

Find nan index

In [2]:
def find_nan_index(lst):
    
    for i, element in enumerate(lst):
        
        if m.isnan(element):
            return i
        
    return len(lst)

## Opti problem

#### Objective function

In [3]:
def objective(x):  #m, n depends on the path: temperary

  S = 0
  
  for i in range(zones_num): # m: present the number of places where we will put our edge
    e_i = region[pg.Rcount(i)]  # elevation of the zone i

    for j in range(zones_num): # n: present the number of places that we will prevet water flow from thanks to the edge
      S += x[i][j] * (slr - e_i)  # maybe direcly ndiro og.
  
  return S

#### Constraint Function

In [ ]:
# It works just for one asset
def constraint(x): 
    
    a = []

    for asset in pg.assets:
      road = T_Ci[asset]
      
      for k in range (len(road)):  # Nombre des lignes
          S = 0

          for i in range (road[k][1], road[k][find_nan_index(road[k])]): # Z3ma nbre de colonne li non nulle 'a définir plus tard'
            for j in range(road[k][0], road[k][find_nan_index(road[k]) - 1]):
              S += x[i][j]

          if S < 1 :   # for each k: which mean each road   
            return False  # If one of the roads is False, we immediatly return false

    return True
    